In [2]:
#Inspired by https://github.com/maetshju/gsoc2018

In [7]:
using Knet: param, param0
using DSP

using FileIO: load, save, loadstreaming, savestreaming
import LibSndFile

using MFCC
using Plots
using BSON

In [10]:
#print(pwd()*"/Datasets/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav")

FRAME_LENGTH = 0.025 # ms
FRAME_INTERVAL = 0.010 # ms

0.01

In [1]:
#1 and 3
onehotencoding(x) = (x==1) ? UInt8.([1;0]) : UInt8.([0;1])

encode1_3(x) = (x==1) ? 1 : 2

classes1_3(x) = (x==1) ? 1 : 2

classes1_3 (generic function with 1 method)

In [3]:
#2 and 3
onehotencoding(x) = (x==2) ? UInt8.([1;0]) : UInt8.([0;1])

encode2_3(x) = (x==2) ? 1 : 2

classes2_3(x) = (x==2) ? 1 : 2

classes2_3 (generic function with 1 method)

classes1_3 (generic function with 1 method)

In [4]:
replaceClasses(x) = classes2_3(x)

replaceClasses (generic function with 1 method)

In [5]:
#Step 2 
"""
    getFeatures( wavName )
Extracts Mel filterbanks and associated labels from `wavName`, fortunately ravdess incorporates labels into the filename
"""
function getFeatures( wavName )
    
    samps, sr = load(wavName)
    samps = vec(samps)
    
    #.+ eps to avoid Inf's and NaN's
    samps.+=eps()

    #Kind of handles resampling! Consider lowering quality of training data if accuracy is too off
    mfccs, _, _ = mfcc(samps, sr, :rasta; wintime=FRAME_LENGTH, steptime=FRAME_INTERVAL)
    
    #Directories might be missing on Repo
    #GET 06!!!
    #emotion = 03-01-06-01-02-01-12.wav
    emotion = parse(Float64,(wavName[82:83]))
    
    #plot(mfccs)
    #print(mfccs)
    
    
    #We're using a Recurrent Network
    
    #print()
    
    #13 Features
    #features = mfccs
    
    #use this if you want to use deltas as additional features
    mfccDeltas = deltas(mfccs, 2)
    features = hcat(mfccs, mfccDeltas)
    #println(size(features))
    
    #print(features[1])
    
    
    #labels = fill( onehotencoding(emotion), size(features)[1] )
    labels = fill( replaceClasses(emotion), size(features)[1] )
    
    
    toReturn = (Float32.(features), UInt8.(labels)) 
    return toReturn
end

#typeof(getFeatures("Datasets/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav"))

getFeatures

In [8]:
"""
    createData(data_dir, out_dir)
Extracts data from files in `data_dir` and saves results in `out_dir`.
"""
function createData(data_dir, out_dir)
    #println("WARNING NOT SAVING")
    for (root, dirs, files) in walkdir(data_dir)
        #println(files)
        
        wavFiles = [x for x in files if occursin("wav",x)]
        
        one_dir_up = basename(root)
        #println(wavFiles)
                    
        for wavFile in wavFiles
            wavPath = joinpath(root, wavFile)
            #println(wavPath)
            #return
            
            x, y = getFeatures(wavPath)
                        
            #x = Float32.(x)
            #println(summary(x))
            #y = UInt8.(replaceClasses.(y))
            #println(summary(y))
                        
            
            #classes = [n for n in 1:8]
                        
            #break
            #print(classes)
                        
            #FOR FLUX
            #y = onehotbatch(y, classes)'
            #no onehot encoding this time
            #y = 
            
            #FOR KNET
                        
            #Keep as is
                        
            base, _ = splitext(wavFile)
            #print(one_dir_up * "-" * base * ".bson")
            dat_name = one_dir_up * "-" * base * ".bson"
            dat_path = joinpath(out_dir, dat_name)
            #println(dat_path)
                        
            #println(y)
            #return
            BSON.@save dat_path x y
        end
    end
end

createData

In [12]:
@time createData("Datasets/ravdess-emotional-speech-audio","Datasets/ravdess-26-features-2-3")

 94.329808 seconds (320.81 M allocations: 29.725 GiB, 6.94% gc time)


In [10]:
#createData(TRAINING_DATA_DIR, TRAINING_OUT_DIR)
#createData(TEST_DATA_DIR, TEST_OUT_DIR)

In [88]:
T = getFeatures("Datasets/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_12/03-01-03-01-02-01-12.wav")

(348, 26)


(Float32[-143.93388 -15.416435 … -0.89059263 -0.38944596; -131.18521 -15.478297 … -0.8413835 -1.1602328; … ; -97.957535 -8.081757 … 1.8839778 0.82921773; -96.52068 -8.230402 … 1.090286 2.860385], UInt8[0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02  …  0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02])

### TO DO IN PREPROCESS:

1. Change Features to be a Array{Float32, 2} (Done?)
2. Label Classes Properly (Also Done)

UndefVarError: UndefVarError: gpu not defined